In [1]:
# imports
import numpy as np
import pandas as pd
import os
import json
import requests

# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [2]:
toronto_bike_df = pd.read_csv('C:/Users/John/Documents/LHL Lecture Material/Stats-Model-Project/data/toronto_bike_df.csv')
toronto_bike_df['ll'] = toronto_bike_df['lat'].astype(str) + ',' + toronto_bike_df['lon'].astype(str)
toronto_bike_df

,lat,lon,num_of_bikes,ll
0,43.665269,-79.319796,19,"43.665269,-79.319796"
1,43.671420,-79.445947,15,"43.67142,-79.445947"
2,43.666224,-79.317693,23,"43.666224,-79.317693"
3,43.653236,-79.376716,24,"43.653236,-79.376716"
4,43.663722,-79.380288,13,"43.663722,-79.380288"
...,...,...,...,...
648,43.772445,-79.511912,15,"43.772445,-79.511912"
649,43.653359,-79.365023,11,"43.653359,-79.365023"
650,43.706436,-79.251044,11,"43.70643614257429,-79.25104443353288"
651,43.655972,-79.370638,15,"43.655972,-79.370638"


In [70]:
def fsquare_stations(coordinates):
    """Put in coordinates to get general information about address within 1000m"""
    url = 'https://api.foursquare.com/v3/places/search'
    categories = 19046 #Metro Station ##13065 #dining and drinking restaurant
    foursquare_key = os.environ["FOUR_SQUARE_API"]
    params = {'ll': coordinates,
            'radius': 1000,
            'categories': categories,
            'limit': 50,
            'fields': 'name,location,rating,stats,price,popularity' }
    headers = {'Accept': 'application/json', 'Authorization': foursquare_key}
    response = requests.get(url, params=params, headers=headers).json()
    return response

In [ ]:
lst_toronto_stations = []
for location in toronto_bike_df['11']:
    lst_toronto_stations.append(fsquare_stations(location))

In [56]:
fsquare_stations('43.65629959857465,-79.38041385515406')

{'results': [{'location': {'address': '300 Yonge St',
    'country': 'CA',
    'cross_street': 'under Dundas St',
    'formatted_address': '300 Yonge St (under Dundas St), Toronto ON M5G 2B3',
    'locality': 'Toronto',
    'postcode': 'M5G 2B3',
    'region': 'ON'},
   'name': 'Dundas Subway Station',
   'popularity': 0.999068987989945},
  {'location': {'address': '449 University Ave',
    'country': 'CA',
    'cross_street': 'at Dundas St. W.',
    'formatted_address': '449 University Ave (at Dundas St. W.), Toronto ON M5G 1W8',
    'locality': 'Toronto',
    'postcode': 'M5G 1W8',
    'region': 'ON'},
   'name': 'St Patrick Subway Station',
   'popularity': 0.995661484033144},
  {'location': {'address': '250 University Ave',
    'country': 'CA',
    'cross_street': 'at Queen St. W.',
    'formatted_address': '250 University Ave (at Queen St. W.), Toronto ON M5H 3E5',
    'locality': 'Toronto',
    'postcode': 'M5H 3E5',
    'region': 'ON'},
   'name': 'Osgoode SUBWAY Station',
   'p

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [65]:
### Used for restaurants
def fsquare_stations_info(coordinates):
    """Put in coordinates to get a dictionary with information about the name,address,ratings and total ratings within 1000m"""
    url = 'https://api.foursquare.com/v3/places/search'
    categories = 13065 #dining and drinking restaurant
    foursquare_key = os.environ["FOUR_SQUARE_API"]
    params = {'ll': coordinates,
            'radius': 1000,
            'categories': categories,
            'limit': 50,
            'fields': 'name,location,rating,stats,price,popularity'}
    headers = {'Accept': 'application/json', 'Authorization': foursquare_key}
    response = requests.get(url, params=params, headers=headers).json()
    restaurant_dict ={'name':[], 'address': [], 'rating': [], 'total_photos': [], 'total_ratings': [], 'num_of_res': [], 'coordinates': []}
    for restaurant in response['results']:
        restaurant_dict['name'].append(restaurant['name'])
        restaurant_dict['address'].append(restaurant['location']['address'])
        restaurant_dict['coordinates'].append(coordinates)
        restaurant_dict['num_of_res'].append(len(response['results']))
        if 'rating' in restaurant:
            restaurant_dict['rating'].append(restaurant['rating'])
        else:
            restaurant_dict['rating'].append(np.nan)
        if 'stats' in restaurant:
            if 'total_photos' in restaurant['stats']:
                restaurant_dict['total_photos'].append(restaurant['stats']['total_photos'])
            else: 
                restaurant_dict['total_photos'].append(0)
            if 'total_ratings' in restaurant['stats']:
                restaurant_dict['total_ratings'].append(restaurant['stats']['total_ratings'])
            else:
                restaurant_dict['total_ratings'].append(0)
        else:
            restaurant_dict['total_photos'].append(0)
            restaurant_dict['total_ratings'].append(0)
    return restaurant_dict

In [25]:
## Metro stations
# def fsquare_stations_info(coordinates):
#     """Put in coordinates to get a dictionary with information about the name, address and number of subways within 1000m"""
#     url = 'https://api.foursquare.com/v3/places/search'
#     categories = 19046 
#     foursquare_key = os.environ["FOUR_SQUARE_API"]
#     params = {'ll': coordinates,
#             'radius': 1000,
#             'categories': categories,}
#     headers = {'Accept': 'application/json', 'Authorization': foursquare_key}
#     response = requests.get(url, params=params, headers=headers).json()
#     subway_dict ={'name':[], 'address': [], 'num_of_stations': [], 'coordinates': []}
#     if len(response['results']) == 0:
#         subway_dict['name'].append(np.nan)
#         subway_dict['address'].append(np.nan)
#         subway_dict['num_of_stations'].append(0)
#         subway_dict['coordinates'].append(coordinates)
#     else:
#         for subway in response['results']:
#             if 'address' not in subway['location']:
#                 subway_dict['address'].append(np.nan)
#             else:
#                 subway_dict['address'].append(subway['location']['address'])
#             subway_dict['name'].append(subway['name'])
#             subway_dict['num_of_stations'].append(len(response['results']))
#             subway_dict['coordinates'].append(coordinates)
#     return subway_dict

In [19]:
ex1 = fsquare_stations_info('43.663722,-79.380288')
ex1

{'name': ['Wellesley Subway Station',
  'Dundas Subway Station',
  'Bloor-Yonge Subway Station',
  "Queen's Park Subway Station"],
 'address': ['551 Yonge St',
  '300 Yonge St',
  '20 Bloor St E',
  '505 University Ave'],
 'num_of_stations': [4, 4, 4, 4],
 'coordinates': ['43.663722,-79.380288',
  '43.663722,-79.380288',
  '43.663722,-79.380288',
  '43.663722,-79.380288']}

In [66]:
lst_toronto_stations_info = []
for location in toronto_bike_df['ll']:
    lst_toronto_stations_info.append(fsquare_stations_info(location))

Put your parsed results into a DataFrame

In [67]:
lst_fsquare_df = []
for data in lst_toronto_stations_info:
    lst_fsquare_df.append(pd.DataFrame(data))


In [101]:
fsquare_df = pd.concat(lst_fsquare_df)
fsquare_df.to_csv('C:/Users/John/Documents/LHL Lecture Material/Stats-Model-Project/data/fsquare_df.csv', index=False)
fsquare_df

,name,address,rating,total_photos,total_ratings,num_of_res,coordinates
0,Rorschach Brewing,1001 Eastern Ave,8.7,157.0,50.0,50.0,"43.665269,-79.319796"
1,The Sidekick,1374 Queen St E,8.8,106.0,35.0,50.0,"43.665269,-79.319796"
2,Lake Inez,1471 Gerrard St E,8.9,31.0,43.0,50.0,"43.665269,-79.319796"
3,Udupi Palace,1460 Gerrard St E,8.6,65.0,107.0,50.0,"43.665269,-79.319796"
4,Godspeed Brewery,242 Coxwell Ave,8.5,206.0,61.0,50.0,"43.665269,-79.319796"
...,...,...,...,...,...,...,...
45,Popeye's Louisiana Kitchen,645 Yonge St,6.5,17.0,11.0,50.0,"43.666874,-79.381155"
46,Pita Land,102 Gerrard St E,6.5,9.0,15.0,50.0,"43.666874,-79.381155"
47,Manulife Centre,55 Bloor St W,6.3,73.0,101.0,50.0,"43.666874,-79.381155"
48,Boston Pizza,401 Yonge St,6.3,11.0,12.0,50.0,"43.666874,-79.381155"


In [97]:
dup_fsquare_df = fsquare_df.copy()
dup_fsquare_df.drop(['num_of_res','coordinates'], axis=1, inplace=True)
dup_fsquare_df.drop_duplicates(inplace=True)
dup_fsquare_df.shape

(2939, 5)

In [98]:
dup_fsquare_df.sort_values('rating', ascending=False)


,name,address,rating,total_photos,total_ratings
17,Byblos,11 Duncan St,9.3,163.0,337.0
9,Northern Belle,913 Dundas St W,9.3,44.0,74.0
1,SOMA chocolatemaker,443 King St W,9.3,254.0,312.0
0,Honest Weight,2766 Dundas St W,9.2,19.0,48.0
1,Porchetta & Co,545 King St W,9.2,59.0,166.0
...,...,...,...,...,...
24,MELTwich Food,704 the Queensway,NaN,0.0,0.0
4,Pizzaville,989 Murray Ross Pky,NaN,0.0,0.0
9,Tri-Star Chinese Restaurant,4724 Jane St,NaN,0.0,0.0
11,Little Caesars Pizza,4710 Jane St,NaN,0.0,0.0


# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [71]:
def yelp_stations(coordinates):
    """Put in coordinates to get information about the name,address,ratings,stats, price and popularity within 1000m"""
    url = 'https://api.yelp.com/v3/businesses/search'
    categories = 'restaurants,All' #dining and drinking restaurant
    yelp_key = os.environ['yelp_key']
    params = {'latitude': coordinates.split(',')[0],
            'longitude': coordinates.split(',')[1],
            'radius': 1000,
            'limit': 50,
            'categories': categories,}
    headers = {'Accept': 'application/json', 'Authorization': "Bearer " + yelp_key}
    response = requests.get(url, params=params, headers=headers).json()
    return response

In [ ]:
yelp_list = []
for place in toronto_bike_df['ll']:
    yelp_list.append(yelp_stations(place))
yelp_list

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [77]:
def yelp_station_details(coordinates):
    """Put in coordinates to get information about the name,address,ratings,stats, price and popularity within 1000m"""
    url = 'https://api.yelp.com/v3/businesses/search'
    categories = 'restaurants,All' #dining and drinking restaurant
    yelp_key = os.environ['yelp_key']
    params = {'latitude': coordinates.split(',')[0],
            'longitude': coordinates.split(',')[1],
            'radius': 1000,
            'limit': 50,
            'categories': categories,}
    headers = {'Accept': 'application/json', 'Authorization': "Bearer " + yelp_key}
    response = requests.get(url, params=params, headers=headers).json()
    yelp_station_details = {'name': [], 'address': [], 'yelp_rating': [], 'yelp_total_ratings': [], 'yelp_num_res': [], 'coordinates': []}
    for locations in response['businesses']:
        yelp_station_details['name'].append(locations['name'])
        yelp_station_details['address'].append(locations['location']['display_address'][0])
        yelp_station_details['yelp_rating'].append(locations['rating'])
        yelp_station_details['yelp_total_ratings'].append(locations['review_count'])
        yelp_station_details['yelp_num_res'].append(len(response['businesses']))
        yelp_station_details['coordinates'].append(coordinates)
    return yelp_station_details

In [78]:
list_yelp_details = []
for location in toronto_bike_df['ll']:
    list_yelp_details.append(yelp_station_details(location))

Put your parsed results into a DataFrame

In [79]:
yelp_df_list = []
for location in list_yelp_details:
    yelp_df_list.append(pd.DataFrame(location))

In [80]:
yelp_df = pd.concat(yelp_df_list)
yelp_df.to_csv('C:/Users/John/Documents/LHL Lecture Material/Stats-Model-Project/data/yelp_df.csv', index=False)

In [91]:
dup_yelp_df = yelp_df.copy()
dup_yelp_df.drop(['yelp_num_res','coordinates'], axis=1, inplace=True)

In [92]:
dup_yelp_df.drop_duplicates(inplace=True)
dup_yelp_df.shape

(2861, 4)

# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

Yelp provided me with more information about the number of restaurants nearby. This is most likely due to the nature of YELP and how customers are able to interact with one another and relay information about new restaurants for everyone to see. Foursquare on the otherhand had information like photos taken which is a neat feature that is not given by YELP even though YELP is known for a forum where people posts photos and share their reviews. YELP's rating only goes up to 5, whereas Foursquare has a rating system up to 10. This gives Foursquare an edge as it provides a more concise rating than YELP. 

Get the top 10 restaurants according to their rating

In [93]:
dup_yelp_df.sort_values(['yelp_rating','yelp_total_ratings'],ascending=False).head(10)

,name,address,yelp_rating,yelp_total_ratings
0,Zeal Burgers,1926 Weston Road,5.0,139.0
1,Grandma Loves You,1084 Yonge Street,5.0,71.0
28,Mallo,785 Bathurst Street,5.0,44.0
35,Gurume Sushi,45 Carlton Street,5.0,39.0
2,9 Bars Cafe Latino Bar,46 St. Clair Avenue E,5.0,36.0
49,Hawker,291 Augusta Ave,5.0,34.0
19,Stock In Trade,307 Danforth Avenue,5.0,30.0
0,Plan B,2943A Bloor Street W,5.0,30.0
24,Egg Sunrise Grill,1422 Dundas Street W,5.0,26.0
34,Jjin Toast,792 Bathurst Street,5.0,24.0


In [99]:
dup_fsquare_df.sort_values(['rating','total_ratings'], ascending=False).head(10)

,name,address,rating,total_photos,total_ratings
17,Byblos,11 Duncan St,9.3,163.0,337.0
1,SOMA chocolatemaker,443 King St W,9.3,254.0,312.0
9,Northern Belle,913 Dundas St W,9.3,44.0,74.0
18,Pai,18 Duncan St,9.2,515.0,932.0
7,Bellwoods Brewery,124 Ossington Ave,9.2,1095.0,639.0
1,Porchetta & Co,545 King St W,9.2,59.0,166.0
6,Koerner Hall,273 Bloor St W,9.2,125.0,81.0
0,Honest Weight,2766 Dundas St W,9.2,19.0,48.0
2,Steam Whistle Brewing,255 Bremner Blvd,9.1,1225.0,886.0
0,Bar Isabel,797 College St,9.1,231.0,392.0


In [100]:
dup_yelp_df[dup_yelp_df['name'] == 'Byblos']

,name,address,yelp_rating,yelp_total_ratings
12,Byblos,11 Duncan Street,4.5,890.0
